# **Transformer Challenge**

1. Get the the notebook from https://keras.io/examples/nlp/neural_machine_translation_with_transformer/
2. Make it run. Note that it uses the spa-eng file from the Anki site
3. Include code to save the model on disk so that you can use the pre-trained model
4. Include code to use the pre-trained embeddings from Stanford.
Link at https://nlp.stanford.edu/projects/glove/
5. Include code to show the layer activations as the ones shown the notebook that shown during the lecture.
Code at https://github.com/tensorflow/text/blob/master/docs/tutorials/transformer.ipynb
6. Work with the model to improve its performance. Things to try are:
* Use more than 30 epochs
* Change the number of ngrams
* Change the learning rate
* Change the optimizer
* Change the metric
* Explore how to use the BLUE (Bilingual Evaluation Understudy)
* Explore how to use the Rouge  score

OPTIONAL: Get and run the code from https://keras.io/examples/nlp/neural_machine_translation_with_keras_hub/
to use the Rouge metric

Write a short report (5 pages) describing your work, results, comments
Deadline: 03/22/2025 @ Noon, CDMX Time, using the Github page :
https://github.com/camachojua/diplomado-ia/tree/main/python/src/student_submissions/Transformer

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [3]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng_extracted" / "spa-eng" / "spa.txt"

2638744/2638744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#Analizando los datos
Cada línea contiene una oración en inglés y su correspondiente oración en español. La oración en inglés es la secuencia de origen y la oración en español es la secuencia objetivo. Anteponemos el token "[start]" y añadimos el token "[end]" a la oración en español.

with open(text_file) as f:

In [4]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [5]:
for _ in range(5):
    print(random.choice(text_pairs))

('Can I stay here tonight?', '[start] ¿Puedo pasar aquí la noche? [end]')
("I'm shorter than him.", '[start] Soy más pequeño que él. [end]')
("I don't like the way she laughs.", '[start] No me gusta la manera en la que ella ríe. [end]')
('I wish I were a good singer.', '[start] Quisiera ser un buen cantante. [end]')
('Tom has lied to you.', '[start] Tom te ha mentido. [end]')


In [6]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


#Vectorizando los datos de texto

In [7]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    ngrams=4,####<-----n-grams
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [8]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [9]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


#Construyendo el modelo

In [10]:
import keras.ops as ops


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        inputs, encoder_outputs = inputs
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [11]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

#Entrenando el modelo

In [12]:
from keras.optimizers import Adam

epochs = 6  # This should be at least 30 for convergence

transformer.summary()
custom_optimizer = Adam(learning_rate=0.001)####<------Tasa de aprendizaje para ADAM
transformer.compile(
    custom_optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"],
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding      │ (None, None, 256)      │      3,845,120 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_1    │ (None, None, 256)      │      3,845,120 │ decoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      3,155,456 │ positional_embedding[… │
│ (TransformerEncoder)      │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_1 (NotEqual)    │ (None, None)           │              0 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder       │ (None, None, 256)      │      5,259,520 │ positional_embedding_… │
│ (TransformerDecoder)      │                        │                │ transformer_encoder[0… │
│                           │                        │                │ not_equal_1[0][0],     │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, None, 256)      │              0 │ transformer_decoder[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, None, 15000)    │      3,855,000 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 108s 69ms/step - accuracy: 0.0776 - loss: 5.4451 - val_accuracy: 0.1291 - val_loss: 3.7411
Epoch 2/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 124s 62ms/step - accuracy: 0.1384 - loss: 3.6498 - val_accuracy: 0.1715 - val_loss: 2.8906
Epoch 3/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 79s 61ms/step - accuracy: 0.1775 - loss: 2.8296 - val_accuracy: 0.1980 - val_loss: 2.4332
Epoch 4/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 83s 62ms/step - accuracy: 0.2031 - loss: 2.3007 - val_accuracy: 0.2096 - val_loss: 2.1870
Epoch 5/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 82s 62ms/step - accuracy: 0.2200 - loss: 1.9647 - val_accuracy: 0.2187 - val_loss: 2.0458
Epoch 6/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 81s 63ms/step - accuracy: 0.2327 - loss: 1.7102 - val_accuracy: 0.2200 - val_loss: 1.9942


# 4. Modificaciones a la clase `PositionalEmbedding` para cargar los embeddings de GloVe, se crea la clase `PositionalEmbeddingGlove`

## Descripción
Se realizaron modificaciones a la clase `PositionalEmbedding` para que no solo maneje las representaciones posicionales, sino también para cargar embeddings preentrenados de GloVe. Esto mejora la calidad de las representaciones de las palabras al aprovechar los vectores de palabras preentrenados de un modelo de embeddings de palabras como GloVe.

## Modificaciones
1. **Carga de los embeddings de GloVe**:
   - Los embeddings de GloVe fueron cargados desde un archivo de texto que contiene los vectores preentrenados.
   - Estos vectores fueron almacenados en un diccionario, donde las claves son las palabras y los valores son sus representaciones vectoriales.

2. **Integración de los embeddings de GloVe en la capa de embedding**:
   - En lugar de usar un embedding aleatorio, se cargó una matriz de embeddings preentrenados y se asignó a la capa de embeddings de la clase `PositionalEmbedding`.
   - Solo las palabras que estén en el vocabulario de entrada serán representadas con los embeddings de GloVe. Las palabras fuera del vocabulario son representadas con un vector de ceros o con un valor de inicialización predeterminado.


In [13]:
class PositionalEmbeddingGlove(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, embedding_matrix=None, **kwargs):
        super().__init__(**kwargs)

        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embed_dim,
            weights=[embedding_matrix] if embedding_matrix is not None else None,
            trainable=False if embedding_matrix is not None else True
        )

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )

        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

# Cargar y usar los embeddings preentrenados de GloVe

## Descripción
En esta celda se cargan los embeddings preentrenados de GloVe y se integran en la red neuronal. Los embeddings de GloVe son utilizados para representar las palabras en el vocabulario antes de pasar por el modelo Transformer.

## Pasos realizados

1. **Carga de los embeddings de GloVe**:
   - Se carga el archivo de GloVe (`glove.6B.50d.txt`) que contiene vectores preentrenados de palabras en un diccionario llamado `embeddings_index`. Cada palabra se mapea a su correspondiente vector de características.

2. **Creación de la matriz de embeddings**:
   - Se construye una matriz llamada `embedding_matrix`, la cual tiene un tamaño igual al del vocabulario de entrada. Cada palabra en el vocabulario se mapea al vector de GloVe correspondiente si está presente en el diccionario `embeddings_index`.

3. **Modificación del modelo**:
   - Se crea una capa personalizada llamada `PositionalEmbeddingGlove`, la cual recibe la matriz de embeddings de GloVe para ser utilizada en el modelo.
   - El encoder y decoder del modelo Transformer ahora utilizan esta capa de embeddings para representar las palabras de manera más informada.

In [14]:
# Ruta al archivo GloVe
glove_path = "glove.6B.50d.txt"

# Cargar GloVe en un diccionario
embeddings_index = {}
with open(glove_path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # Primera palabra en la línea
        coefs = np.asarray(values[1:], dtype="float32")  # Resto son los valores del embedding
        embeddings_index[word] = coefs

print(f"Se cargaron {len(embeddings_index)} palabras preentrenadas.")

spa_vocab = spa_vectorization.get_vocabulary()
vocab_size = len(spa_vocab)  # Número de palabras en el vocabulario
embedding_dim = 50  # Tamaño del embedding de GloVe

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for i, word in enumerate(spa_vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector  # Usar embedding preentrenado


embed_dim = 50 # Cambiamos al tamaño del embedding de GloVe para armar el modelo
latent_dim = 2048
num_heads = 8

# Crea la capa PositionalEmbedding con la matriz de embeddings preentrenados
embedding_layer = PositionalEmbeddingGlove(
    sequence_length, vocab_size, embed_dim, embedding_matrix=embedding_matrix
)

# Construcción del encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = embedding_layer(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

# Construcción del decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = embedding_layer(decoder_inputs)  # Usar la misma capa de embedding
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# Crear el modelo completo
transformer2 = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

Se cargaron 130220 palabras preentrenadas.


Con esto podemos compilar el modelo llamado transformer2 en el cual la capa de positional embedding usa los datos preentrenados de https://nlp.stanford.edu/projects/glove/

In [15]:
epochs = 6

transformer2.summary()
transformer2.compile(
    "rmsprop",
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"],
)
transformer2.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_glo… │ (None, None, 50)       │        751,000 │ encoder_inputs[0][0],  │
│ (PositionalEmbeddingGlov… │                        │                │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_2 (NotEqual)    │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_1     │ (None, None, 50)       │        288,348 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_3 (NotEqual)    │ (None, None)           │              0 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_1     │ (None, None, 50)       │        369,698 │ positional_embedding_… │
│ (TransformerDecoder)      │                        │                │ transformer_encoder_1… │
│                           │                        │                │ not_equal_3[0][0],     │
│                           │                        │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, None, 50)       │              0 │ transformer_decoder_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, None, 15000)    │        765,000 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,174,046 (8.29 MB)

 Trainable params: 1,424,046 (5.43 MB)

 Non-trainable params: 750,000 (2.86 MB)

Epoch 1/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 52s 29ms/step - accuracy: 0.0596 - loss: 6.4812 - val_accuracy: 0.0950 - val_loss: 4.8059
Epoch 2/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.1015 - loss: 4.9523 - val_accuracy: 0.1254 - val_loss: 4.2418
Epoch 3/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.1198 - loss: 4.5819 - val_accuracy: 0.1406 - val_loss: 4.0138
Epoch 4/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.1310 - loss: 4.3914 - val_accuracy: 0.1464 - val_loss: 3.8773
Epoch 5/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.1378 - loss: 4.2646 - val_accuracy: 0.1528 - val_loss: 3.7953
Epoch 6/6
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.1419 - loss: 4.1972 - val_accuracy: 0.1569 - val_loss: 3.7511


In [16]:
%%capture
!pip install nltk rouge-score

# Métricas BLEU y ROUGE

Las métricas BLEU y ROUGE son utilizadas para evaluar la calidad de los sistemas de traducción automática, comparando las traducciones generadas por el modelo con las traducciones de referencia.

### BLEU (Bilingual Evaluation Understudy)
- **BLEU** mide la precisión n-grama de una traducción, comparando la cantidad de n-gramas coincidentes entre la traducción generada y las referencias.
- Se utiliza un **suavizado** para evitar ceros en los cálculos de n-gramas raros o ausentes.
- El cálculo de BLEU se realiza usando la función `corpus_bleu` de la librería `nltk`.

### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
- **ROUGE** evalúa la calidad de la traducción basándose en la comparación de n-gramas y subsecuencias con las referencias.
- Se considera principalmente el **recall** y la **longitud L** (ROUGE-L), que mide la longitud máxima de la subsecuencia común más larga.
- Se calcula usando la librería `rouge_score`.


In [17]:
#Definir la función de decodificación
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer({
            "encoder_inputs": tokenized_input_sentence,
            "decoder_inputs": tokenized_target_sentence
        })
        sampled_token_index = ops.convert_to_numpy(ops.argmax(predictions[0, i, :])).item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

# Métricas BLEU y ROUGE
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np

nltk.download('punkt')  #Descargar tokenizer para BLEU

def calculate_bleu(pairs, model, n_samples=100):
    references = []
    hypotheses = []

    selected_pairs = random.sample(pairs, min(n_samples, len(pairs)))

    for eng, spa in selected_pairs:
        #Decodificar la traducción del modelo
        translated = decode_sequence(eng)

        #Procesar referencias e hipótesis
        ref = spa.replace('[start]', '').replace('[end]', '').strip().split()
        hyp = translated.replace('[start]', '').replace('[end]', '').strip().split()

        references.append([ref])
        hypotheses.append(hyp)

    #Calcular BLEU con suavizado para evitar ceros
    smoothie = SmoothingFunction().method4
    return corpus_bleu(references, hypotheses, smoothing_function=smoothie)

def calculate_rouge(pairs, model, n_samples=100):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rougeL_scores = []

    selected_pairs = random.sample(pairs, min(n_samples, len(pairs)))

    for eng, spa in selected_pairs:
        translated = decode_sequence(eng)
        ref = spa.replace('[start]', '').replace('[end]', '').strip()
        hyp = translated.replace('[start]', '').replace('[end]', '').strip()

        scores = scorer.score(ref, hyp)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return np.mean(rouge1_scores), np.mean(rougeL_scores)

#Evaluación con las métricas
print("\nEvaluación de métricas de traducción:")
print(f"BLEU score: {calculate_bleu(test_pairs, transformer, n_samples=300)}")
rouge1, rougeL = calculate_rouge(test_pairs, transformer, n_samples=300)
print(f"ROUGE-1: {rouge1:.4f}, ROUGE-L: {rougeL:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



Evaluación de métricas de traducción:
BLEU score: 0.1341316843687854
ROUGE-1: 0.5559, ROUGE-L: 0.5478


###Resultados 6.2 n-grams
Solo modificamos *_eng_vectorization_* y agregamos _*ngrams=m_* para algun $m\in\mathbb{N}$.

| Número de n-grams | Accuracy | Loss  | Val Accuracy | Val Loss |
|-------------------|----------|-------|-------------|----------|
| 2                 | 0.2392   | 2.0617 | 0.2311      | 2.2354   |
| 4                 | 0.2361   | 2.1264 | 0.2285      | 2.2466   |
| 6                 | 0.2354   | 2.1343 | 0.2301      | 2.2569   |

Por lo tanto, tomamos $n-grams=2$ ya que es el que nos dio un mayor valor de validación en Accuracy.

###Resultados 6.3 tasa de aprendizaje
Continuamos con la tasa de aprendizaje $lr$, agregamos el optimizador ADAM para modificar su tasa de aprendizaje:

| ADAM(lr)          | Accuracy | Loss  | Val Accuracy | Val Loss |
|-------------------|----------|-------|-------------|----------|
| lr=0.001          | 0.2289   | 1.7962 | 0.2208      | 2.0142   |
| lr=0.005          | 0.2051   | 2.1636 | 0.2031      | 2.2623   |
| lr=0.01           | 0.0502   | 5.9670 | 0.0499      | 5.8368   |

Podemos notar como a medida de que aumentamos la tasa de aprendizaje del optimizador ADAM, la precisión disminuye y tambien aumenta la perdida del modelo. Esto último ocurre tanto en el entrenamiento como en la validación. Por lo tanto, se deben de balancear estas dos métricas como es el caso de la tasa de aprendizaje de $lr=0.001$ o $lr=0.005$ donde valores cercanos a esos valores darían mejores resultados tanto en la Accuracy como en la perdida, dependiendo que se quiera mejorar más.


###6.4 Optimizador
Cambiamos el optimizador cargando algun otro de keras y rn *_compile_* agregamos estos optimizadores los cuale son los siguientes:

| Optimizador(lr)| Accuracy | Loss  | Val Accuracy | Val Loss |
|-------------------|----------|-------|-------------|----------|
| RMSprop(0.001)    |0.0503   | 5.8545 | 0.0504      | 5.7475   |
| AdamW(0.001)         | 0.2481   | 1.3703 | 0.2293      | 1.8646   |
| Lion           | 0.0590   | 5.6973 | 0.0599      | 5.5801   |

Nuevamente, la variacion de ADAM, ADAMW fue el que obtuvo mejores resultados con errores de entrenamiento más bajos que el ADAM normal.

###6.5 Métrica
Al cambiar la métrica a *sparse categorical accuracy*, esta baja utilizando el ADAM(lr=0.001) de la siguiente manera:

| Loss   | Sparse Categorical Accuracy | Val Loss | Val Sparse Categorical Accuracy |
|--------|-----------------------------|----------|-----------------------------------|
| 5.6287 | 0.0605                      | 5.5408   | 0.0601                           |

Por lo cual no es conveniente cambiarla ya que aumenta la perdida.

###6.6 y 6.7 Metricas BLUE (Bilingual Evaluation Understudy) y Rouge
# Métricas de Evaluación para Traducción Automática/Texto Generado

## **BLEU (Bilingual Evaluation Understudy)**
- Mide la similitud entre texto generado y referencias humanas usando *coincidencia de n-gramas* (usualmente 1-4 gramas).
- **Usos típicos**:
  - Evaluación de traducción automática
  - Sistemas de conversión texto-a-texto
- **Limitaciones**:
  - No considera significado semántico
  - Pobre desempeño con idiomas flexibles
  - Favorece traducciones literales

## **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**
-Mide la adecuación del contenido mediante *recuperación de unidades léxicas*.
- **Usos típicos**:
  - Evaluación de resúmenes
  - Sistemas de generación de texto
- **Limitaciones**:
  - No evalúa coherencia
  - Sensible a redundancias
  - Ignora relaciones semánticas

| Característica          | BLEU                      | ROUGE               |
|-------------------------|---------------------------|---------------------|
| Enfoque principal       | Precisión                 | Recall              |
| Ideal para              | Traducción                | Resumen/Generación  |
| Sensibilidad a longitud | Penaliza cortos           | Menos sensible      |
| Unidad de análisis      | N-gramas exactos          | Subsecuencias       |
| Tiempo de cálculo       | Rápido                    | Moderado            |
| Métrica oficial en      | WMT (traducción)          | DUC (resumen)       |

Pra interpretar los scores:
- **BLEU**:
  - mayor a 0.4: Excelente
  - 0.3-0.4: Bueno
  - menor a 0.2: Pobre
- **ROUGE**:
  - mayor a 0.5: Alta calidad
  - 0.3-0.5: Aceptable
  - menor a 0.2: Baja calidad

Para nuestro ejemplo de 6 epocas con un optimizador Adam con *lr=0.001* y *ngrams=2* obtuvimos:
- BLEU score:  0.1341316843687854.
- ROUGE-L: 0.5478.

Donde los modelos son pobres debido a las pocas epocas que se usaron.

In [18]:
#Guardar el modelo
transformer.save("modelo_transformer.keras")

#Guardar los pesos
transformer.save("transformer_pesos.h5")

# Guardar modelo con pretrained embedding
transformer2.save("transformer_model_glove.keras")

transformer = keras.models.load_model(
    "transformer_model_rnd.keras",
    custom_objects={
        "encoder_inputs": encoder_inputs,
        "decoder_inputs": decoder_inputs,
        "TransformerEncoder": TransformerEncoder,
        "TransformerDecoder": TransformerDecoder,
        "PositionalEmbedding": PositionalEmbedding,
    },
)

transformer_glove = keras.models.load_model(
    "transformer_model_glove.keras",
    custom_objects={
        "encoder_inputs": encoder_inputs,
        "decoder_inputs": decoder_inputs,
        "TransformerEncoder": TransformerEncoder,
        "TransformerDecoder": TransformerDecoder,
        "PositionalEmbeddingGlove": PositionalEmbeddingGlove,
    },
)

#Decoding test sentences

In [19]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer(
            {
                "encoder_inputs": tokenized_input_sentence,
                "decoder_inputs": tokenized_target_sentence,
            }
        )

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]

In [20]:
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

print(input_sentence)
print(translated)

It looks like I'm not needed here.
[start] parece que no tengo que no esté aquí [end]
